In [2]:
library(rvest)

Warning message:
"패키지 'rvest'는 R 버전 4.1.3에서 작성되었습니다"


In [5]:
base_url <- 'https://movie.naver.com/movie/bi/mi/point.nhn?code='
code = 191613
target_url <- paste0(base_url, code)
html <- read_html(target_url)

title_unclean <- html %>%
    html_nodes("title") %>%
    html_text()

title <- substr(title_unclean, 1, unlist(gregexpr(' : 네이버 영화', title_unclean))-1)
exist <- 
    html %>% 
    html_nodes('dl[class=info_spec]') %>% 
    html_nodes('dt') %>% 
    html_text()

steps <- html %>%
    html_nodes('dl[class=info_spec]') %>%
    html_nodes('dd')

# 개요, 감독, 출연, 등급 중 결측값이 존재하는 경우를 대비함
step1 = NA
step2 = NA
step3 = NA
step4 = NA
for (i in 1:length(exist)){
    if (exist[i] == '개요()'){
        step1_unclean <- steps[i] %>%
            html_nodes('p') %>%
                html_nodes('span') %>%
            html_text()
        step1 <- gsub('\\t|\\n|\\r', '', step1_unclean)
    }else if (exist[i] == '감독'){
        step2 <- steps[i] %>%
            html_nodes('p') %>%
            html_text()
    }else if (exist[i] == '출연'){
        step3 <- steps[i] %>%
            html_text()
    }else if (exist[i] == '등급'){
        step4_unclean <- steps[i] %>%
            html_nodes('p') %>%
            html_text()
        step4 <- gsub('\\t|\\n|\\r', '', step4_unclean)
    }
}

tdt <- html %>%
    html_nodes('div[class=viewing_graph]')

# 성별, 나이별 관람추이가 존재하지 않는 경우 관람객 통계가 존재하지 않으므로 결측값으로 처리함
if (length(tdt) == 0){
    audience_age_10 <- NA
    audience_age_20 <- NA
    audience_age_30 <- NA
    audience_age_40 <- NA
    audience_age_50 <- NA
    audience_score <- NA
    audience_count <- NA
    audience_male <- NA
    audience_female <- NA
    audience_10 <- NA
    audience_20 <- NA
    audience_30 <- NA
    audience_40 <- NA
    audience_50 <- NA
} else {
    audi_age <- html %>%
        html_nodes('strong[class=graph_percent]') %>%
        html_text()
    audience_age_10 <- audi_age[1]
    audience_age_20 <- audi_age[2]
    audience_age_30 <- audi_age[3]
    audience_age_40 <- audi_age[4]
    audience_age_50 <- audi_age[5]

    audience_score <- html %>% 
        html_nodes('div[class=grade_audience]') %>%
        html_nodes('div[class=star_score]') %>%
        html_nodes('em') %>%
        html_text() %>% paste(collapse='')

    audience_count <- html %>% 
        html_nodes('div[class=grade_audience]') %>%
        html_nodes('span[class=user_count]') %>%
        html_nodes('em') %>%
        html_text() %>% paste(collapse='')

    audience_male <- (html %>%
        html_nodes('div[class=graph_area]') %>%
        html_nodes('div[class=grp_male]') %>%
        html_nodes('strong[class=graph_point]') %>%
        html_text())[2]

    audience_female <- (html %>%
        html_nodes('div[class=graph_area]') %>%
        html_nodes('div[class=grp_female]') %>%
        html_nodes('strong[class=graph_point]') %>%
        html_text())[2]

    audience_age <- html %>%
        html_nodes('div[class=grp_age]') %>%
        html_nodes('strong[class=graph_point]') %>%
        html_text()

    audience_10 <- audience_age[6]
    audience_20 <- audience_age[7]
    audience_30 <- audience_age[8]
    audience_40 <- audience_age[9]
    audience_50 <- audience_age[10]

}

netizen_score <- html %>% 
    html_nodes('div[class=grade_netizen]') %>%
    html_nodes('div[class=star_score]') %>%
    html_nodes('em') %>%
    html_text() %>% paste(collapse='')

netizen_count <- html %>% 
    html_nodes('div[class=grade_netizen]') %>%
    html_nodes('span[class=user_count]') %>%
    html_nodes('em') %>%
    html_text() %>% paste(collapse='')

ntz_male <- (html %>%
    html_nodes('div[class=graph_area]') %>%
    html_nodes('div[class=grp_male]') %>%
    html_nodes('strong[class=graph_point]') %>%
    html_text())[1]

ntz_female <- (html %>%
    html_nodes('div[class=graph_area]') %>%
    html_nodes('div[class=grp_female]') %>%
    html_nodes('strong[class=graph_point]') %>%
    html_text())[1]

ntz_age <- html %>%
    html_nodes('div[class=grp_age]') %>%
    html_nodes('strong[class=graph_point]') %>%
    html_text()
ntz_10 <- ntz_age[1]
ntz_20 <- ntz_age[2]
ntz_30 <- ntz_age[3]
ntz_40 <- ntz_age[4]
ntz_50 <- ntz_age[5]

In [6]:
step1

[1] "애니메이션, 코미디, 가족" "스페인, 영국"            
[3] "96분 "                    " 2019.11.15 개봉"

In [7]:
step2

[1] "서지오 파블로스, 카를로스 마르티네즈 로페즈"

In [8]:
step3

[1] "제이슨 슈왈츠먼, J.K. 시몬스, 라시다 존스더보기"

In [9]:
step4

[1] "[국내] 전체 관람가"